```sql

with has_funded as (
select user_id as stash_user_id,
       true as has_funded
from integrated_users.universal_transfers
where source='external'
group by 1),

last_lock_event as (
select user_id as stash_user_id,
       max_by(event_type,applied_at) as last_lock_event
from source_pg_main.lock_events
group by 1),

locked_reason as (
select
  id as stash_user_id,
  restricted_reason as locked_reason
from source_pg_main.users),

email_active as (
select
  stash_user_uuid,
  true as email_active
from mart.fct_braze_event 
where
  event_at >= current_date - interval '365 days' and
  event_name in
    ('email_clicked','email_opened','email_sent','inapp_clicked','push_opened')
group by 1),

email_undeliverable as (
select
  stash_user_uuid,
  true as email_undeliverable
from mart.fct_braze_event 
where event_name in ('email_bounce','email_spam','email_unsubscribed')
group by 1),

is_pre_subscriber_2018 as (
select
                                             stash_user_uuid,
  coalesce(not ever_linked,true)          as never_linked,
  coalesce(not has_funded,true)           as never_funded,
  coalesce(last_lock_event='UNLOCK',true) as not_locked,
                                             locked_reason,
                                             email_active,
                                             email_undeliverable
from
  mart.dim_user left join
  mart.external_money_links using (stash_user_id) left join
  has_funded using (stash_user_id) left join
  last_lock_event using (stash_user_uuid) left join
  locked_reason using (stash_user_uuid))

select *,
  never_linked and never_funded and not_locked
    as is_pre_subscriber_2018
from is_pre_subscriber_2018
```